In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from datetime import datetime, timedelta

from bs4 import BeautifulSoup 
import re
import urllib.request, urllib.parse, urllib.error

In [2]:
driver=webdriver.Chrome(executable_path="./chromedriver_win32/chromedriver")
driver.get("https://techcrunch.com/")

In [3]:
for i in range(3):
    wait=WebDriverWait(driver, 100)
    load=wait.until(EC.presence_of_element_located((By.XPATH, '//span[contains(@class, "gradient-text gradient-text--green-gradient")]')))
    load.click()
wait=WebDriverWait(driver, 10)
load=wait.until(EC.presence_of_element_located((By.XPATH, '//span[contains(@class, "gradient-text gradient-text--green-gradient")]')))

In [4]:
titles=driver.find_elements_by_class_name('post-block__title')
dates=[]
print(len(titles))
for i in range(len(titles)):
    titles[i]=titles[i].find_element_by_class_name('post-block__title__link').get_attribute('href')

120


In [5]:
news=[]
for title in titles:
#     Using Selenium
    li=[]
    driver.get(title)
    
    wait=WebDriverWait(driver, 20)
    load=wait.until(EC.presence_of_element_located((By.XPATH, '//time[contains(@class, "full-date-time")]')))
    pastdate=datetime.now()-timedelta(hours=2)
    utcdiff=datetime.now()-datetime.utcnow()
    newsdate=datetime.strptime(driver.find_element_by_class_name('full-date-time').get_attribute('datetime'), '%Y-%m-%dT%H:%M:%S')+utcdiff
    
    if newsdate<pastdate:
        continue
    
    li.append(driver.find_element_by_class_name('article__title').text)
    li.append(title)
    
    try:
        li.append(driver.find_element_by_class_name('article__featured-image').get_attribute('src'))
    except:
        li.append('')
        
    content=driver.find_element_by_class_name('article-content').find_elements_by_xpath("./*")
    li.append(content[0].text)
    s=""
    for i in range(1, len(content)):
        s=s+content[i].text+'\n'
    li.append(s)
    li.append(newsdate)
    li.append('Technology')    
    
    news.append(li)

#     Using Bs4
#     li=[]
#     h=urllib.request.urlopen(title)
#     file=h.read()
#     soup=BeautifulSoup(file, 'html')
#     li.append(soup.find('h1', attrs={'class': 'article__title'}).get_text().replace(u'\xa0', u' ').strip())
#     li.append(title)
#     try:
#         li.append(soup.find('img', attrs={'class': 'article__featured-image'})['src'])
#     except:
#         li.append('')
#     content=soup.find('div', attrs={'class': 'article-content'}).find_all('p')
#     li.append(content[0].get_text().replace(u'\xa0', u' ').strip())
#     d=soup.find('div', attrs={'class': 'article__byline__meta'}).get_text()
#     print(li)
#     print(d)11
#         li.append()
#         news.append(li)

In [6]:
df=pd.DataFrame(news, columns=['Title', 'Link', 'Image Link', 'Summary', 'Content', 'Date', 'Genre'])
df

,Title,Link,Image Link,Summary,Content,Date,Genre
0,MFS Africa collects $100M to expand its digita...,https://techcrunch.com/2021/11/10/mfs-africa-g...,https://techcrunch.com/wp-content/uploads/2021...,Investor interest in African fintech continues...,The company confirmed to TechCrunch that it ha...,2021-11-10 13:32:03,Technology
1,Amenitiz wants to upgrade the tech stack of in...,https://techcrunch.com/2021/11/10/amenitiz-wan...,https://techcrunch.com/wp-content/uploads/2021...,"Meet Amenitiz, a Barcelona-based startup that ...",The company just raised a $7.5 million (€6.5 m...,2021-11-10 13:30:59,Technology
2,Kenyan edtech startup Craydel raises $1 millio...,https://techcrunch.com/2021/11/09/kenyan-edtec...,https://techcrunch.com/wp-content/uploads/2021...,"Kenyan edtech startup Craydel, a platform for ...","The round was led by Enza Capital, a private v...",2021-11-10 12:30:44,Technology


In [7]:
df.to_csv('techcrunch.csv', encoding='utf-8-sig')

In [8]:
driver.close()
driver.quit()